In [1]:
import planet


In [2]:
aoi = {
    "type":
    "Polygon",
    "coordinates": [[[37.40796, 122.24337], [37.40023, 122.24442],
                     [37.39993, 122.23222], [37.40893, 122.23235],]]
}

# The item IDs we wish to order
images = ['20180423_182027_102c']


In [3]:
# Google Earth Engine configuration
cloud_config = planet.order_request.google_earth_engine(
    project='searsville-wetlands-melguo236', collection='searsvillesample')
# Order delivery configuration
delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

In [4]:
# Product description for the order request
data_products = [
    planet.order_request.product(item_ids=images,
                                 product_bundle='analytic_sr_udm2',
                                 item_type='PSScene')
]

# Build the order request
order = planet.order_request.build_request(name='order',
                                                products=data_products,
                                                delivery=delivery_config)

print(order)

{'name': 'order', 'products': [{'item_ids': ['20180423_182027_102c'], 'item_type': 'PSScene', 'product_bundle': 'analytic_sr_udm2'}], 'delivery': {'google_earth_engine': {'project': 'searsville-wetlands-melguo236', 'collection': 'searsvillesample'}}}


In [6]:
async def create_and_deliver_order(order_request, client):
    '''Create and deliver an order.

    Parameters:
        order_request: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_request)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'],
                          callback=reporter.update_state,
                          max_attempts=0)

    # Grab the details of the orders
    order_details = await client.get_order(order_id=order['id'])

    return order_details

In [7]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(order, client)

03:30 - order 3187c82b-4cdc-4e38-85c3-3fd11c5b8165 - state: success


In [8]:
print(order_details['_links']['results'][0])


{'delivery': 'success', 'expires_at': '2023-08-06T21:24:32.955Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2OTEzNTcwNzIsInN1YiI6ImFRbkpLeTJZSFhETWpQTFo0QjdQaHRmSGN6T1BqUllZcUtrT1FMT0JEeFNHVDBOM2JRRkgwVFA4Kzkza3VWYTU2ZGI1SG9TNWNTV2NHTmIrekNibFFnPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS8yMDE4MDQyM18xODIwMjdfMTAyY18zQl9BbmFseXRpY01TX1NSLnRpZj9YLUdvb2ctQWxnb3JpdGhtPUdPT0c0LVJTQS1TSEEyNTZcdTAwMjZYLUdvb2ctQ3JlZGVudGlhbD1jb21wdXRlLWdjcy1zdmNhY2MlNDBwbGFuZXQtY29tcHV0ZS1wcm9kLmlhbS5nc2VydmljZWFjY291bnQuY29tJTJGMjAyMzA4MDUlMkZhdXRvJTJGc3RvcmFnZSUyRmdvb2c0X3JlcXVlc3RcdTAwMjZYLUdvb2ctRGF0ZT0yMDIzMDgwNVQyMTI0MzJaXHUwMDI2WC1Hb29nLUV4cGlyZXM9ODYzOTlcdTAwMjZYLUdvb2ctU2lnbmF0dXJlPTIzY2FhZjE1MGRlZTA4NGM4NTU2MmY1YzE5OTYwZjlhNTA5NTk1YmJlZjE3ZTlhODhjZmZiYmU1NTc5M